In [ ]:
'''
Social network visualization w/ code adapted from https://earlyprint.org/jupyterbook/metadata.html
'''
import os,re
import matplotlib.pyplot as plt
import pandas as pd
def authors(csv):
    df = pd.read_csv(csv)
    authors = df['author']
    ids = df['id']
    numFiles = len(ids)
    count = 0
    dict = {}
    while count < numFiles:
        words = set(authors[count].split(';'))
        # removing unnecessary keywords
        words.discard('')
        # Removing unnecessary dates  
        newWords = []
        for w in words: 
            w = w.replace('.','')
            # w = re.sub(r'\([^)]*\)','',w)
            if re.search('printer',w):
                continue
            w = re.sub(r' ca|-|[0-9]{4}|, Sir,|, Sir|fl|, d|,  A profession of faith|, Mr','',w)
            w = re.sub(r', Earl of,  or 8|, ca|,  aut|, \?|,  Of schisme|, 16th/17th cent|, attributed name','',w)
            w = re.sub(r',   aut|, bachelor of civil law| Tragedie of Darius| Monarchick tragedies|Doomesday| Paraenesis to the Prince|,  A profession of faith','',w)
            w = w.strip('|,|, ')
            newWords.append(w)
        dict[ids[count]] = list(set(newWords))
        count += 1
    return dict 

In [ ]:
'''
Create python tuples: (id, authors)
'''
csv = '/srv/data/CSVs/virginia.csv'
info = authors(csv)
edgelist = [] # Create an empty list      
# Add each printers name to the edgelist
for id in info.keys():
    for name in info[id]:
        edgelist.append((id, name))

In [ ]:
import networkx as nx # Import networkx

B = nx.Graph() # Create an empty graph object

# Create lists of our two node types
ids = []
auth = []
for e in edgelist:
    ids.append(e[0])
    auth.append(e[1])
# Make sure names or IDs don't repeat
texts = list(set(ids))
auth = list(set(auth))
# Add each node list to our graph, with bipartite and group attributes to keep track of which is which
B.add_nodes_from(texts, bipartite="text", group=1)
B.add_nodes_from(auth, bipartite="author(s)", group=2)

# Add edge list to our graph
B.add_edges_from(edgelist)

In [228]:
from pyvis.network import Network # Import pyvis

# Create an empty pyvis graph
g = Network(width=800,height=800,notebook=True,heading='')

# Load our networkx graph into pyvis
for id,auth in edgelist:
    g.add_node(id, id, title=id, color='purple')
    g.add_node(auth, auth, title=auth, color='pink')    
    g.add_edge(id,auth)
g.show("virginiaAuthors.html")